# Basic image manipulation

In this notebook we will showcase how we can use the array indexing and array math to perform basic image manipulation using OpenCV.

We can start by importing required packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [ ]:
# make the plots bigger
plt.rcParams["figure.figsize"] = (10,10)

We can then define some functions we will use very often.

In [ ]:
def open_image(imgpath, mode=cv2.IMREAD_GRAYSCALE):
    """
    Open an image as an numpy array.
    
    Parameters
    ----------
    imgpath : `str`
        Path to the image.
    mode : `int`, optional
        Mode with which to open the image, i.e. color vs black and white.
        Must be one of the OpenCV recognized constants such as 
        `cv2.IMREAD_GRAYSCALE` or `cv2.IMREAD_COLOR`
        
    Returns
    --------
    img : `np.array`
        Image.
        
    Raises
    ------
    OSError - when the file could not be opened.
    """
    img = cv2.imread(imgpath, mode)
    if img is None:
        raise OSError("Can't open file:", imgpath)
    else:
        return img

def show(img, ax=None, **kwargs):
    """
    Show image using matplotlib.
    
    Parameters
    ----------
    img : `np.array`
        Image to display.
    ax : `matplotlib.pyplot.Axes` or `None`, optional
        Ax on which to plot the image. If  no axis is given
        a new figure is created.
    kwargs : `dict`, optional
        Keyword arguments that are passed to `imshow`.
    """
    if ax is None:
        fig, ax = plt.subplots()
        
    ax.imshow(img, **kwargs)

To break down what is happening here a bit - for those less familiar with Python syntax, we defined a could of functions: `open_image` and `show`.

So far we wrote all our code in a cell in a notebook and then executed it. Writing flat code like that, code that executes exactly how you read it line by line, is referred to procedural coding. There's nothing necessarily wrong with doing that, but in cases when there is a lot of code to write and many little procedures to string together it often pays off to generalize them and organize them in small units. This helps us not repeat code over and over again. That way, when we want to change something later, we can change it in that one "packaged" up place and it will affect the whole codebase. In procedural coding this would not be the case - we would need to change the functionality every time it appears. 

Functions are these little packages of code that we can repeat multiple times. In python they are defined by using the keyword `def` which is followed by the function name. This means that the code that follows, belongs to this function and not the general code that is executed every time you run the cell, only when you **call** the function. To make the distinction of which code belongs to the function and which doesn't you need to indent it. 

The `function_name` is followed by parenthesis `()` and a colon `:`. The parenthesis usually contain the function arguments. Functions do not *have* to have arguments, but they usually do. Python will usually assign the values you give to function in the order you define them, f.e. for a `def function(arg1, arg2):` that is called as `function(1, 2)` Python will assume you meant`arg1=1` and `arg2=2`. These are called positional arguments. Sometimes, however, you might want to have some values that user is not expected to provide because there exist reasonable default values you can assume for them. In that case you can provide these values in the function declaration like `def function(arg1, arg2=2):`. We can also provide full argument names for our function as well, in which case we call them keyword arguments. For example, calling function `def function(arg1, arg2):` as `function(arg2=2, arg1=1)` would still assign `arg1=1` and `arg2=2` despite the reverse order we gave them in. Notice this is different than the example with positional arguments above.

Following the function declaration is probably Python's best feature - documentation. We use the [NumPy Doc](https://numpydoc.readthedocs.io/en/latest/format.html) style documentation which means we have, usually, a one short line describing what the function does (not how it's implemented!), a section `Parameters` with a list of function arguments, their `types` and a short description and we usually have a `Returns` section describing the returned value and its `type`. Not all functions need to have all, or any, of these sections and some might warrant having additional ones, f.e. `Raises` when they raise an error.

After the documentation follows the function definition, the bit of code that does something with the given input and, usually, that consist of some kind of a calculation of values based on the function input and we want to give that calculated value back to the user at the end - ergo we `return` the value(s) at the end of the function.

So put all together:
```
def function_name(arg1, arg2):
    """Short description of function purpose.
    
    Parameters
    ----------
    arg1 : `type`
        Short description of this argument
    arg2 : `type`
        Short description of this argument
        
    Returns
    --------
    val : `type`
        Short description of the returned value(s)
    
    Notes
    -----
    BTW, calling this function will also place a random order on Amazon
    
    Examples
    --------
    **VERY** useful to list some sometimes
    
    Raises
    ------
    Error - description of when this error is raised.
    """
    # do something with the values you were given
    result = arg1 + arg2
    
    return result
```

Let's get back on track now.

# How do real images look like

Let's take a peek. We can inspect the array dimensions, data type and data value ranges.

For examples we can use the images in the `images` directory.

In [ ]:
imgbw = open_image("images/cat.png")
show(imgbw, cmap="gray")

In [ ]:
print("shape     ", imgbw.shape)
print("data type ", imgbw.dtype)
print("max       ", imgbw.max())
print("min       ", imgbw.min())
print("mean      ", imgbw.mean())
print("Image element: ", imgbw[0, 0])

Let's do the same, but let's just see what happens if we use a different image mode. 

In [ ]:
imgc = open_image("images/cat.png", cv2.IMREAD_UNCHANGED)
show(imgc)

In [ ]:
print("shape     ", imgc.shape)
print("data type ", imgc.dtype)
print("max       ", imgc.max())
print("min       ", imgc.min())
print("mean      ", imgc.mean())
print("Image element: ", imgc[0, 0])

Do you notice any differences between the two examples?          
Is the image still black and white?   
Can you spot differences between the way the two images look like?        
Are our array elements the same?           
Our dtypes?          

Can we trust what our eyes see?

## Doing image math

Even basic operations on images, such as subtracting, can be complicated. At the same time they can be quite powerful. 

Let's start by applying the basic array math we just used in 1_numpy_arrays.       
Adding a value to the whole array increases brightness of the image. Subtracting value does the opposite.

In [ ]:
show(imgc+90, cmap="gray")

Slicing operators return image regions.

In [ ]:
dimx, dimy = imgbw.shape
centerx, centery = int(dimx/2), int(dimy/2)

In [ ]:
# square selection
box_size = 200
cutout = imgbw[
    centerx-box_size:centerx+box_size, 
    centery-box_size:centery+box_size
]
show(cutout, cmap="gray")

How would **you** use numpy indexing to select a circle?

Note: scrolling down might spoil the answer.

In [ ]:
x, y = np.ogrid[:dimx, :dimy]
mask = ( (x-centerx)**2 + (y-centery)**2 ) > 200**2
cutout = imgbw.copy()
cutout[mask] = 0
show(cutout, cmap="gray")

OpenCv helps us a lot here by having a bunch of these methods already available which saves us a headache.

**Note** - the way NumPy an OpenCV index arrays is not exactly the same. Above we could confidently state:
```
x, y = np.ogrid[:dimx, :dimy]
```
but with OpenCV the origin is not bottom left, it is the top left instead. 
```
cv2.circle(mask, (centery, centerx), 200, (255, 255, 255), -1)
```
Using top left as origin is a well established convention in Computer Vision, so in this case it's `matplotlib` that's "odd". Naturally Matplotlib uses bottom left, because it's meant to be used as a plotting tool - not an image viewer.

In [ ]:
mask = np.zeros(imgbw.shape, dtype=np.uint8)
cv2.circle(mask, (centery, centerx), 200, 255, -1)
cutout = cv2.bitwise_and(imgbw, mask) #or img = img & mask
show(cutout, cmap="gray")

Can you figure out what all the elements in the function mean?

How about in the ellipse example?

In [ ]:
mask = np.zeros(imgbw.shape, dtype=np.uint8)
sma, smi = 200, 150
rot_angle = 0
froma, toa = 0, 360
cv2.ellipse(mask, (centery, centerx), (sma, smi), rot_angle, froma, toa, (255, 255, 255), -1)
cutout = cv2.bitwise_and(imgbw, mask) #or img = img & mask
show(cutout, cmap="gray")

Note that we have a lot of power when drawing with `cv2`. We don't even have to worry about image boundaries.

In [ ]:
mask = np.zeros(imgbw.shape, dtype=np.uint8)
sma, smi = 300, 400
rot_angle = 45
froma, toa = 270, 360
cv2.ellipse(mask, (centery-200, centerx), (sma, smi), rot_angle, froma, toa, (255, 255, 255), -1)
cutout = cv2.bitwise_and(imgbw, mask) #or img = img & mask
show(cutout, cmap="gray")

## Doing some **useful** math to our images.

Lets look at one example where we can "improve" our original image by applying just basic operations on it.

In [ ]:
img = open_image("images/son1.png")
show(img, cmap="gray")

Obviously, the image itself is not unreadable. However the picture of the sonnet is not the best.         
There are obvious issues with the light gradient across the image that impedes our ability to read the text itself.

I have taken a liberty of creating a synthetic background for this image that we can use to improve the readability of the text.

In [ ]:
img2 = open_image("images/son2.png")
show(img2, cmap="gray")

How would you use the second image to improve on the first?

Ok, let's try subtraction.

In [ ]:
diff = img - img2
show(diff, cmap="gray")

Looks a little bit more uniform but what's with the sudden blotches?

Annoyingly, the math on images doesn't always work as you would expect.          
Let's take an example of two numpy arrays with a single element of type `uint8`:

In [ ]:
a = np.array([1], dtype=np.uint8)
b = np.array([2], dtype=np.uint8)

What do you think the following line will print?

In [ ]:
a - b

Can you figure out why?

What would you do to the first, or second image to improve on this?

In [ ]:
diff[diff < 5] = 255
show(diff, cmap="gray")

Is this the best we can do with subtraction? 

Is this the best we can do in general?      

Think about it a bit, what is it we want? We know that, in real life, the paper is the same color top right and bottom left. It just appears on the image it's not. So we want the top right, where it's very bright "white", and bottom left, where it's dark "black" to become the same color.          I.e. we want to "put in more white" where it's black and no white where it's already white. What mathematical operation would normalize these different values to the same one?

If you figured it out - great job! If not, what about division?

On face value, dividing the images shouldn't suffer from the same overflow - so we won't need to manually add in data to fix it.

Lets think about couple of extreme cases here:
* What happens if we divide two very bright areas? 
* What happens with very dark areas?

In [ ]:
div = img/img2
show(div, cmap="gray")

Let's check out some values on this image though.

In [ ]:
print("max", div.max())
print("min", div.min())
print("avg", div.mean())

So how come this image displays correctly?

Once again, `cv2` has a lot of functionality to help us re-normalize our images, however, they might not always behave in the most intuitive ways because of the `uint8` overflow. A lot of the times, `matplotlib` and/or `cv2` will perform this step under the hood before displaying the image to us. This also means you can't always trust the things you see - because there's extra math that happens. 

Sometimes, however, `matplotlib` and `cv2` do get confused. In that case I usually manage to fix my woes by using the `convertScaleAbs` function from OpenCV. We can see it's documentation by running `help` built-in function or by relying on a bit of Jupyter Notebook magic and adding a question mark at the end.

In [ ]:
cv2.convertScaleAbs?

Now an interesting question might be - what has this lesson actually taught us and will we have to do something similar to what we did with the poem to work on astronomical images? 

The answer is somewhat complicated. Partly - yes - what we did with the scan of Poem for Lena is kind of what happens with astronomical images. The particular step where we take away the effects of non-uniformly illuminated focal plane in particular is called flat-fielding. But unlike here, where I approximated the gradient in the image by slapping some gradients in numpy array, in a properly calibrated scientific instrument these effects are rigorously measured. 

What are all the effects that are removed before we even show you the image, and the procedures of addition/subtraction/divisions/multiplications/derivations etc. that are used to do that are all explained in great details in [Learn Astropy Guide to CCD Data Reduction](http://www.astropy.org/ccd-reduction-and-photometry-guide/v/dev/notebooks/01-00-Understanding-an-astronomical-CCD-image.html).            
In the tutorials they will first artificially create all of the effects astronomical images suffer from, then simulate a perfect astronomical image without them, then add all of these effects into a fake realistic astronomical image. But that's not all! Then they will take that simulated realistic image and work the other way around to remove all the effects from it and end up with a well calibrated image.         
I *very* much recommend **leafing** through the guide to get familiar with astronomical data. It is not required you repeat all the exercises and demonstrations within, but if you pay attention to the text and the examples - you will see they do pretty much a similar thing to what we did here.

However, our hope is to already work with calibrated images. The calibration procedure have some generic steps, but they are also subject to the particulars of individual instrument, and we don't often build two of the same ones, meaning each one is a little bit different. So the answer is also partly - no. We will hopefully not have to calibrate our exposures as the AstroPy examples do. 

We will still work with data however. Data that we can make conscious purposeful quality cuts on. It is not difficult, therefore, to imagine that we might need to rescale our image (involves subtraction and division of the whole image), or, in a more targeted approach, I can imagine us wanting to select areas around bright stars in order to make clever local background estimation and subtraction on etc. We will see more of this in following notebooks.

In [ ]:
# as an example of how to create a simple gradient
x, y = np.ogrid[:dimx, :dimy]
mask = ( (x-centerx)**2 + (y-centery)**2 )
show(mask)

# Summary

* Matplotlib is a plotting utility that we can use to visually inspect our images
   * pretty cool
* OpenCV is a library that contains many many helpful functions to manipulate and visualize images
* Images are represented as arrays of different dimensions       
   * Black and white images are just arrays of numbers. The size of the array are the dimensions of the image, and the pixel values represent the pixel brightness in that point.
   * Color images have 3 or 4 numbers for each of the pixel. The first 3 values represent the brightness of the pixel, but this time for the particular color they represent, and the 4th value is opacity/alpha/transparency.
   * These values are usually only discrete integer values in some range. More precisely, for images most commonly shown on our screens, it's 8 bit unsigned integers in the [0, 255] range.
* But this is not always the case        
   * Scientific data often represents real measurement, so restricting it to 255 values represents a large loss of precision.
   * To see our data we often have to squish the full data range into the available range to display the image on the screen. I.e we have to somehow "bin" the data into those 255 boxes. We'll see how we do this and what it means in the next exercise.
* Basic math operations modify some colloquially understood image properties
   * addition increases brightness
   * subtraction decreases it
   * we can remove trends and smooth over details in an image by selecting parts of image          
     or by providing per-pixel values which we can add/subtract or divide out
   * We can express these trends in images as functions over pixels